In [1]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

In [2]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [3]:
TRAINING_DIR = "./trainD"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "./test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 254 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [4]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [5]:
history = model.fit(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Epoch 1/10
19/26 [====================>.........] - ETA: 8s - loss: 1.1582 - acc: 0.3991

C:\Users\ssvis\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


26/26 [==============================] - 37s 1s/step - loss: 1.0664 - acc: 0.4248 - val_loss: 0.7853 - val_acc: 0.5000
INFO:tensorflow:Assets written to: model2-001.model\assets
Epoch 2/10
 4/26 [===>..........................] - ETA: 17s - loss: 0.7074 - acc: 0.5771

C:\Users\ssvis\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


26/26 [==============================] - 24s 885ms/step - loss: 0.6861 - acc: 0.6170 - val_loss: 0.6935 - val_acc: 0.5052
INFO:tensorflow:Assets written to: model2-002.model\assets
Epoch 3/10
 4/26 [===>..........................] - ETA: 16s - loss: 0.6945 - acc: 0.6000

...

26/26 [==============================] - 23s 896ms/step - loss: 0.6654 - acc: 0.6571 - val_loss: 0.7370 - val_acc: 0.5052
Epoch 4/10
26/26 [==============================] - 24s 903ms/step - loss: 0.6172 - acc: 0.6718 - val_loss: 0.5479 - val_acc: 0.7371
INFO:tensorflow:Assets written to: model2-004.model\assets
Epoch 5/10
23/26 [=========================>....] - ETA: 2s - loss: 0.6381 - acc: 0.6722

C:\Users\ssvis\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


26/26 [==============================] - 24s 908ms/step - loss: 0.6352 - acc: 0.6730 - val_loss: 0.6183 - val_acc: 0.6495
Epoch 6/10
26/26 [==============================] - 24s 910ms/step - loss: 0.5523 - acc: 0.7182 - val_loss: 0.4489 - val_acc: 0.7990
INFO:tensorflow:Assets written to: model2-006.model\assets
Epoch 7/10


C:\Users\ssvis\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


26/26 [==============================] - 23s 892ms/step - loss: 0.5509 - acc: 0.7288 - val_loss: 0.5440 - val_acc: 0.7629
Epoch 8/10
26/26 [==============================] - 25s 941ms/step - loss: 0.5622 - acc: 0.7299 - val_loss: 0.4874 - val_acc: 0.7835
Epoch 9/10
26/26 [==============================] - 25s 954ms/step - loss: 0.5553 - acc: 0.7218 - val_loss: 0.8021 - val_acc: 0.5619
Epoch 10/10
26/26 [==============================] - 24s 937ms/step - loss: 0.5537 - acc: 0.8018 - val_loss: 0.7098 - val_acc: 0.5361


In [2]:
import cv2
import numpy as np
import winsound

from keras.models import load_model
model=load_model("./model2-006.model")

results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0) 


haarcascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    label=0
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
        
        
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label],(x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('LIVE',im)
    key = cv2.waitKey(10)
    if label == 0:
        winsound.Beep(2500,100)
    if key == 27: 
        break
cap.release()

cv2.destroyAllWindows()